# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# pip install gmaps

In [3]:
!jupyter nbextension enable --py gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:

file_path = pd.read_csv('../WeatherPy/output/cities.csv')

# Store csv created in part one into a DataFrame
df = pd.DataFrame(file_path)
df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Qinhuangdao,39.9317,119.5883,60.49,19,0,16.89,CN,1620171751
1,1,Jamestown,42.0970,-79.2353,60.80,82,90,5.75,US,1620171298
2,2,Puerto Ayora,-0.7393,-90.3518,80.60,74,20,18.41,EC,1620171302
3,3,Hithadhoo,-0.6000,73.0833,83.53,67,28,3.36,MV,1620171038
4,4,Camacha,33.0833,-16.3333,62.60,59,75,12.66,PT,1620171331


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [7]:
# Heatmap of humidity
locations = df[['Lat','Lng']].astype(float)
humidity = df['Humidity'].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=100,point_radius=4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
new_df = df.loc[(df['Max Temp'] >70) & (df['Humidity'] < 50) & (df['Wind Speed'] < 6),:]
len(new_df)

10

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = new_df[['City','Country','Lat','Lng']].reset_index(drop=True)
hotel_df

,City,Country,Lat,Lng
0,Luk Pu,CN,23.1742,112.2809
1,Jintūr,IN,19.6167,76.7000
2,Centralina,BR,-18.5839,-49.1994
3,Charters Towers,AU,-20.1000,146.2667
4,Qarqīn,AF,37.4185,66.0436
5,Arlit,NE,18.7369,7.3853
6,Laguna,US,38.4210,-121.4238
7,Makkah al Mukarramah,SA,21.4267,39.8261
8,Kosonsoy,UZ,41.2561,71.5508
9,Najrān,SA,17.4924,44.1277


In [12]:
# Add new column for hotel name
hotel_df['Hotel Name'] = ''
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Luk Pu,CN,23.1742,112.2809,
1,Jintūr,IN,19.6167,76.7000,
2,Centralina,BR,-18.5839,-49.1994,
3,Charters Towers,AU,-20.1000,146.2667,
4,Qarqīn,AF,37.4185,66.0436,
5,Arlit,NE,18.7369,7.3853,
6,Laguna,US,38.4210,-121.4238,
7,Makkah al Mukarramah,SA,21.4267,39.8261,
8,Kosonsoy,UZ,41.2561,71.5508,
9,Najrān,SA,17.4924,44.1277,


In [13]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key,
}

# Iterate through 
for index, row in hotel_df.iterrows():

    # get lat, lng from df
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_search = requests.get(base_url, params=params)
    
    # convert to json
    hotel_search = hotel_search.json()

    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index,'Hotel Name'] = hotel_search['results'][0]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [14]:
# Drop rows with missing field/result
hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != ""]
hotel_df

,City,Country,Lat,Lng,Hotel Name
1,Jintūr,IN,19.6167,76.7000,Hotel 19 February
2,Centralina,BR,-18.5839,-49.1994,Hotel Arantes Centralina
3,Charters Towers,AU,-20.1000,146.2667,Commercial Hotel
5,Arlit,NE,18.7369,7.3853,Hôtel Temet
6,Laguna,US,38.4210,-121.4238,Hampton Inn & Suites Sacramento-Elk Grove Lagu...
7,Makkah al Mukarramah,SA,21.4267,39.8261,"Makkah Clock Royal Tower, A Fairmont Hotel"
8,Kosonsoy,UZ,41.2561,71.5508,"Гостиница ""Goldstar'"


In [15]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Add marker layer ontop of heat map
hotel_names = hotel_df['Hotel Name'].tolist()

hotel_marker = gmaps.marker_layer(
    locations,
    info_box_content = hotel_info
)

# Display figure
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(hotel_marker)

fig

Figure(layout=FigureLayout(height='420px'))